# **Linear Algebra for Data Science**: HW3, task 3  
# *Iterative solutions of the linear systems* (3 pts)

### <div align="right"> &copy; Volodymyr Kuchynskyi & Rostyslav Hryniv, 2023 </div>

## Completed by:   
*   First team member
*   Second team member

The aim of this task is to discuss three classic iterative methods of solving linear systems $A\mathbf{x} = \mathbf{b}$, namely, the Jacobi, Gauss-Seidel, and successive over-relaxation (SOR) methods. We will implement them, analyse their convergence, and compare convergence rate with the spectral radius of the corresponding matrix $B$ in the iteration scheme $\mathbf{x}_{k+1} = B\mathbf{x}_k + \mathbf{d}$

In [ ]:
import numpy as np
#Note: other common plotting libraries (such as plotly, altair, etc) or other modules of matplotlib are allowed, if you prefer them. Simply replace the next import with your library of choice
import matplotlib.pyplot as plt

# 1. The methods

A general method to turn a linear system into an iterative one proceeds as follows. We write $A = M - N$ with an invertible $M$ and recast $A\mathbf{x} = \mathbf{b}$ as the iteration scheme
$$M\mathbf{x}_{k+1} = N\mathbf{x}_k + \mathbf{b}, \qquad \text{or} \qquad \mathbf{x}_{k+1} = (M^{-1}N)\mathbf{x}_k + M^{-1}\mathbf{b}$$

All three methods of interest are formulated in terms of the lower-triangular, diagonal, and upper-triangular parts $L$, $D$, and $U$ of $A$, respectively.



## 1.1 Jacobi method
Take $M=D$; then with $B_J:=- D^{-1}(L+U)$ and $\mathbf{d}_J:=D^{-1} \mathbf{b}$ the iterations are
$$
  \mathbf{x}_{k+1} = B_J \mathbf{x}_k + \mathbf{d}_J
$$
or
$$
  x_j^{(k+1)} = a_{jj}^{-1}\bigl( b_j -  \sum_{l\ne j} a_{jl} x^{(k)}_l \bigr)
$$
The method is known to converge if $A$ is <font color='red'>*diagonally dominating*</font>, i.e., if $|a_{jj}|> \sum_{i\ne j}|a_{ji}|$ for all $j=1,2,\dots, n$.



## 1.2 Gauss-Seidel method
Take $M=U+D$; then with $B_{GS}:=- (L+D)^{-1}U$ and $\mathbf{d}_{GS}:= (U+D)^{-1} \mathbf{b}$ the iterations are
$$
  \mathbf{x}_{k+1} = B_{GS} \mathbf{x}_k + \mathbf{d}_{GS}
$$
or
$$
  x_j^{(k+1)} = a_{jj}^{-1}\bigl(b_j - \sum_{l< j} a_{jl} x^{\color{red}{(k+1)}}_l  - \sum_{l > j} a_{jl} x^{(k)}_l\bigr)
$$
The method is known to converge if $A$ is <font color='red'>*positive definite*</font>



## 1.3 Successive over-relaxation method (SOR)
SOR depends on a parameter $\omega \in (0,2)$ and starts with re-writing  $\omega A$ as $\omega A = (D + \omega L) - ((1-\omega)D - \omega U)$, so that $A\mathbf{x} = \mathbf{b}$ is converted into
$$ (D + \omega L) \mathbf{x}  = \bigl((1-\omega)D - \omega U\bigr)\mathbf{x} + \omega\mathbf{b}$$
and becomes the iteration scheme  
$$
  \mathbf{x}_{k+1} = B_{SOR} \mathbf{x}_k + \mathbf{d}_{SOR}
$$
with $B_{SOR} = (D+\omega L)^{-1}((1-\omega) D - \omega U)$ and $\mathbf{d}_{SOR} = \omega(D + \omega L)^{-1}\mathbf{b}$, or   

$$
  x_j^{(k+1)} = \frac\omega{a_{jj}} \Bigl(b_j - \sum_{l< j} a_{jl}x_l^{\color{red}{{(k+1)}}} - \sum_{l> j} a_{jl} x_l^{(k)} \Bigr)+ (1-\omega)x_j^{(k)}
$$
The method is known to converge if $A$ is <font color='red'>*positive definite*</font>; $\omega=1$ coincides with the Gauss-Seidel method but other $\omega\in (0,2)$ can significantly improve the convergence rate.

# 2. Implementation **(1 pt)**

By default, the iterations start with $\mathbf{x}_0:=\mathbf{0}$; they stop when relative increment $\|\mathbf{x}_{k+1} - \mathbf{x}_{k}\|/(\|\mathbf{x}_{k+1}\| + \|\mathbf{x}_{k}\|)$ does not exceed a given threshold $\varepsilon$ or the number of iteration exceeds some $N$. We can take the default values $\varepsilon = 10^{-4}$ and $N=10^4$

## 2.1 Jacobi method

In [ ]:
## implement the function
def jacobi(A, b):
    """Solve the system iteratively using the Jacobi method

    Args:
      A (np.array): a 2D array for coefficient matrix of the system of equations
      b (np.array): a 1D array for the solution vector

    Returns:
      (x, incs) (Tuple(np.array, np.array)): a tuple containing the solution x (if the method converges) and the list of increments ||x_{k+1} - x_k|| for further analysis
    """
    pass

## 2.2 Gauss-Seidel method

In [ ]:
## implement the function
def gs(A, b):
    """Solve the system iteratively using the Gauss-Seidel method

    Args:
      A (np.array): a 2D array for coefficient matrix of the system of equations
      b (np.array): a 1D array for the solution vector

    Returns:
      (x, incs) (Tuple(np.array, np.array)): a tuple containing the solution x (if the method converges) and the list of increments ||x_{k+1} - x_k|| for further analysis
    """
    pass

## 2.3 Successive over-relaxation method

In [ ]:
## implement the function
def sor(A, b, w):
    """Solve the system iteratively using the Gauss-Seidel method

    Args:
      A (np.array): a 2D array for coefficient matrix of the system of equations
      b (np.array): a 1D array for the solution vector
      w : the \omega parameter for the SOR method. See the explanation in markdown cell above

    Returns:
      (x, incs) (Tuple(np.array, np.array)): a tuple containing the solution x (if the method converges) and the list of increments ||x_{k+1} - x_k|| for further analysis
    """
    pass

# 3. Testing convergence **(0.5 pts)**

Generate random column vectors $\mathbf{b}, \mathbf{v}\in \mathbb{R}^{100}$ with independent and uniformly distributed entries in $(0,1)$ and set $A = \mathbf{v}\mathbf{v}^\top$

In [ ]:
v = np.random.rand(100)[:, np.newaxis]
b = np.random.rand(100)
A = v.dot(v.T)

## 3.1 Convergence

Discuss which of the three iteration schemes for $A\mathbf{x} = \mathbf{b}$ will probably converge and which not. Quote the corresponding results

---
**Your explanations here**

---

## 3.2 Testing
Run these iterations to confirm your predictions

In [ ]:
# your code here

# 4. Convergence rates **(1 pt)**

Now change the matrix $A$ to $A=  \alpha I + \mathbf{v}\mathbf{v}^\top$ with a real $\alpha$.

In [ ]:
def create_matrix(alpha):
  return alpha * np.identity(100) + v.dot(v.T)

## 4.1 Convergence
For what $\alpha$ will each of the methods converge? Quote the corresponsing results guaranteeing convergence

---
**Your explanations here**

---

## 4.2 Testing convergence

Run the implemented methods to confirm your guess

In [ ]:
# your code with alpha examples for each method
alpha_1 = #your example value of alpha here
A_1 = create_matrix(alpha_1)
# your code here
alpha_2 = #your example value of alpha here
A_2 = create_matrix(alpha_2)
# your code here
alpha_3 = #your example value of alpha here
A_3 = create_matrix(alpha_3)
# your code here

## 4.3 Convergence rate
For each of the methods (and several $\omega$ for SOR) calculate the <font color='red'>convergence factors</font> $$\rho_{k+1}:=\frac{\|\mathbf{x}_{k+1} - \mathbf{x}_{k}\|}{\|\mathbf{x}_{k} - \mathbf{x}_{k-1}\|}$$ and plot them for available $k$. Compare the limiting value of $\rho_k$ with the <font color='red'>spectral radius</font> of the respective matrix $B$.


In [ ]:
# your code here

# 5. Conculsions **(0.5 pts)**

Summarize in several sentences what you achieved in this task, what were the main obstacles and how you overcome them. You can also add some suggestions how this task can be improved in the future




---
**Your comments come here**

---